In [1]:
import pandas as pd
import numpy as np

In [2]:
import google.datalab.storage as storage
import pandas as pd
from io import BytesIO

mybucket = storage.Bucket('bharti_patel')
data_csv = mybucket.object('merged_data.csv')

uri = data_csv.uri
%gcs read --object $uri --variable data
df = pd.read_csv(BytesIO(data), index_col=0, parse_dates=True, infer_datetime_format=True)
df.head()

,Close,Volume,change_after_3_days,change_after_2_days,change_after_1_days,binary_sentiment_after_3_days,sentiment_after_3_days,binary_sentiment_after_2_days,sentiment_after_2_days,binary_sentiment_after_1_days,sentiment_after_1_days,title_clean
2008-03-31,12262.889648,273610000,1.839884,0.450750,-0.895215,pos,negative,pos,negative,neg,positive,resource warssilver gold the last american her...
2008-04-01,12654.360352,295530000,-5.659022,-10.133497,-2.294553,neg,positive,neg,positive,neg,positive,usa the great depressionventure capitalist fre...
2008-04-02,12608.919922,232760000,-3.177763,-4.816371,0.971870,neg,positive,neg,positive,pos,negative,bush caused the financial market meltdown
2008-04-04,12609.419922,181260000,2.246841,-0.011266,0.018968,pos,negative,neg,neutral,pos,neutral,the death of financial systemfed interest rate...
2008-04-11,12325.419922,286850000,6.365066,5.246454,4.319129,pos,negative,pos,negative,pos,negative,the black death of financial collapserich coun...


In [3]:
x = df.title_clean
y = df.sentiment_after_3_days
z = df.binary_sentiment_after_3_days

In [4]:
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences

tokenizer = Tokenizer()
tokenizer.fit_on_texts(x)
sequences = tokenizer.texts_to_sequences(x)

/usr/local/envs/py3env/lib/python3.5/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


In [6]:

vocab_size = len(tokenizer.word_index) + 1

In [8]:
max_sentlen = max([len(x) for x in sequences])

In [10]:
x_seq = pad_sequences(sequences, maxlen=max_sentlen)
#x_r, x_c = x_seq.shape
#x_seq = x_seq.reshape(1,x_r,x_c)

print('Shape of data tensor:', x_seq.shape)

Shape of data tensor: (1310, 754)


In [11]:
from sklearn.preprocessing import OneHotEncoder
from sklearn.preprocessing import LabelEncoder

In [12]:
enc = LabelEncoder()
onehot = OneHotEncoder(sparse=False)
seq_label = enc.fit_transform(y)
#seq_label = seq_label.reshape(len(seq_label), 1)
#seq_label_hot = onehot.fit_transform(seq_label)
print('Shape of seq_label tensor:', seq_label.shape)
print(seq_label[0])

Shape of seq_label tensor: (1310,)
0


In [14]:
from numpy import asarray
from numpy import zeros
# load the whole embedding into memory
embeddings_index = dict()
f = open('./glove.6B.100d.txt', encoding='utf-8')
for line in f:
    values = line.split()
    word = values[0]
    coefs = asarray(values[1:], dtype='float32')
    embeddings_index[word] = coefs
f.close()


In [15]:
# find the embedding vector dimensions size from the most common word in english: 'the'
# http://www.dictionary.com/e/commonwords/
embedding_size = len(embeddings_index['the'])
print (embedding_size)

100


In [16]:
print('Loaded %s word vectors.' % len(embeddings_index))
# create a weight matrix for words in training docs
print(vocab_size)
embedding_matrix = zeros((vocab_size, embedding_size))
print('Shape of embedding matrix:', embedding_matrix.shape)




Loaded 400000 word vectors.
61211
Shape of embedding matrix: (61211, 100)


In [18]:
from sklearn.model_selection import train_test_split
from keras.utils import to_categorical
n_samples = len(x_seq)
train_pct = 0.8
test_pct = 1 - train_pct
SEED = 2000
seq_label = to_categorical(seq_label)
x_seq_train, x_seq_validation_test, seq_label_train,seq_label_validation_test = train_test_split(x_seq, seq_label, test_size=int(test_pct*n_samples), train_size=int(train_pct*n_samples), random_state=SEED)
x_seq_validation,x_seq_test,seq_label_validation,seq_label_test = train_test_split(x_seq_validation_test,seq_label_validation_test,test_size=.5, random_state=SEED)

#from sklearn.cross_validation import train_test_split
#SEED = 2000
#x_train, x_validation_and_test, y_train, y_validation_and_test = train_test_split(x, y, test_size=.02, random_state=SEED)
#x_validation, x_test, y_validation, y_test = train_test_split(x_validation_and_test, y_validation_and_test, test_size=.5, random_state=SEED)
#x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=.02, random_state=SEED)

print("Training sequences shape:", x_seq_train.shape)
print("Training labels shape:",seq_label_train.shape)
print("Test sequences shape:", x_seq_test.shape)
print("Test labels shape:", seq_label_test.shape)
print("Validation sequences shape:", x_seq_validation.shape)
print("Validation labels shape:", seq_label_validation.shape)

Training sequences shape: (1048, 754)
Training labels shape: (1048, 3)
Test sequences shape: (131, 754)
Test labels shape: (131, 3)
Validation sequences shape: (130, 754)
Validation labels shape: (130, 3)


In [44]:
# define baseline model
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.models import Sequential
from keras.layers.recurrent import LSTM
from keras.layers.core import Activation
from keras.layers.embeddings import Embedding
from keras.layers import TimeDistributed
from keras.layers import Dropout
from keras.layers import Dense
from keras.layers import Bidirectional
from keras.layers import Flatten
import tensorflow as tf
import keras_metrics

In [45]:
def as_keras_metric(method):
    import functools
    from keras import backend as K
    import tensorflow as tf
    @functools.wraps(method)
    def wrapper(self, args, **kwargs):
        """ Wrapper for turning tensorflow metrics into keras metrics """
        value, update_op = method(self, args, **kwargs)
        K.get_session().run(tf.local_variables_initializer())
        with tf.control_dependencies([update_op]):
            value = tf.identity(value)
        return value
    return wrapper


In [46]:
precision = as_keras_metric(tf.metrics.precision)
recall = as_keras_metric(tf.metrics.recall)

In [54]:
# define model
model = Sequential()
e = Embedding(vocab_size, embedding_size, weights=[embedding_matrix], input_length=max_sentlen, mask_zero=True)
#e = Embedding(vocab_size, embedding_size, weights=[embedding_matrix], input_length=max_sentlen)
model.add(e)
model.add(Bidirectional(LSTM(embedding_size, dropout=0.2, recurrent_dropout=0.2)))
model.add(Dense(1))
model.add(Dense(3, activation='sigmoid'))
model.compile(loss='categorical_crossentropy', optimizer='rmsprop', metrics=['accuracy',keras_metrics.precision(), keras_metrics.recall()])
# summarize the model
print(model.summary())


_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_6 (Embedding)      (None, 754, 100)          6121100   
_________________________________________________________________
bidirectional_6 (Bidirection (None, 200)               160800    
_________________________________________________________________
dense_11 (Dense)             (None, 1)                 201       
_________________________________________________________________
dense_12 (Dense)             (None, 3)                 6         
Total params: 6,282,107
Trainable params: 6,282,107
Non-trainable params: 0
_________________________________________________________________
None


In [52]:
# fit the model
model.fit(x_seq_train, seq_label_train, epochs=50, verbose=0,validation_data=(x_seq_validation, seq_label_validation))
# evaluate the model
loss, accuracy, precision, recall = model.evaluate(x_seq_test, seq_label_test, verbose=0)


In [53]:
print('Accuracy: %f, precision: %f, recall: %f, f1_score: %f' % ((accuracy), (precision), (recall), (2 * precision * recall / (precision + recall))))

Accuracy: 0.549618, precision: 0.448322, recall: 0.893130, f1_score: 0.596980
